<a href="https://colab.research.google.com/github/grillinr/evolutionary-computing/blob/main/final/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and seed for easier checking

In [4]:
import random
import os
import argparse
import math
from typing import List, Tuple


import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


SEED = 5173
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")
print(device)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

cuda


# Define helper functions

In [5]:
def prepare_data_with_scaler(data, scaler=None, fit=False):
    data = data.dropna()
    X = data.drop(columns=["id", "record", "type"]).values.astype(np.float32)
    y = data["type"].astype("category").cat.codes.values

    if fit:
        X = scaler.fit_transform(X)
    else:
        X = scaler.transform(X)

    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)


def evaluate(model, X, y, criterion):
    model.eval()
    with torch.no_grad():
        logits = model(X)
        loss = criterion(logits, y)
        y_pred = logits.argmax(dim=1).cpu().numpy()

    y_true = y.cpu().numpy()
    return {
        "loss": loss.item(),
        "accuracy": accuracy_score(y_true, y_pred),
        "f_beta_macro": fbeta_score(y_true, y_pred, average="macro", beta=2, zero_division=0)
    }


def estimate_flops(model, input_shape):
    """
    Estimate FLOPs for Linear and Conv2d layers only.
    Args:
        model (nn.Module): PyTorch model
        input_shape (tuple): shape of one input sample, e.g., (1, 3, 224, 224) or (1, input_dim)
    Returns:
        total_flops (int)
    """
    flops = 0

    def count_layer(layer, x_in, x_out):
        nonlocal flops
        # Conv2d FLOPs = Kx * Ky * Cin * Cout * Hout * Wout
        if isinstance(layer, nn.Conv2d):
            out_h, out_w = x_out.shape[2:]
            kernel_ops = layer.kernel_size[0] * layer.kernel_size[1]
            flops += kernel_ops * layer.in_channels * layer.out_channels * out_h * out_w
        # Linear FLOPs = input_features * output_features
        elif isinstance(layer, nn.Linear):
            flops += layer.in_features * layer.out_features

    hooks = []
    for layer in model.modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            hooks.append(layer.register_forward_hook(count_layer))

    dummy = torch.randn(input_shape).to(next(model.parameters()).device)
    with torch.no_grad():
        model(dummy)

    for h in hooks:
        h.remove()

    return flops

# Create Model Architecture (DNN)

In [6]:
class DNN(nn.Module):
    def __init__(self, input_size=32, hidden=(32, 16, 8), num_classes=5, dropout_rate=0.5):
        super().__init__()
        layers = []
        input_dim = input_size

        for h in hidden:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout_rate))
            input_dim = h

        layers.append(nn.Linear(input_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Main Training loop

In [8]:
# Load data
dataset = pd.read_csv("/content/train.csv")
train_dataset, val_dataset = train_test_split(dataset, train_size=0.7, random_state=SEED)
scaler = StandardScaler()
X_train, y_train = prepare_data_with_scaler(train_dataset, scaler, fit=True)
X_val, y_val = prepare_data_with_scaler(val_dataset, scaler, fit=False)

X_train, y_train = X_train.to(device), y_train.to(device)
X_val, y_val = X_val.to(device), y_val.to(device)

In [9]:
# Configuration
class Hyperparameters:
    def __init__(self, lr, epochs, hidden, dropout_rate, patience):
        self.lr = lr
        self.epochs = epochs
        self.hidden = hidden
        self.dropout_rate = dropout_rate
        self.patience = patience

    def __repr__(self):
        return f"Hyperparameters(lr={self.lr}, epochs={self.epochs}, hidden={self.hidden}, dropout_rate={self.dropout_rate}, patience={self.patience})"

    def __str__(self):
        return self.__repr__()

In [10]:
def train(params: Hyperparameters):
  # Create model
  model = DNN(hidden=params.hidden, dropout_rate=params.dropout_rate).to(device)

  class_counts = train_dataset['type'].value_counts()
  weights = 1.0 / class_counts.values
  weights = torch.FloatTensor(weights).to(device)
  criterion = nn.CrossEntropyLoss(weight=weights)
  optimizer = optim.Adam(model.parameters(), lr=params.lr)

  # Training loop with early stopping
  best_val_loss = float('inf')
  patience_counter = 0
  epochs_run = params.epochs
  for epoch in range(1, params.epochs + 1):
      model.train()
      optimizer.zero_grad()
      out = model(X_train)
      loss = criterion(out, y_train)
      loss.backward()
      optimizer.step()
      train_loss = loss.item()

      train_metrics = evaluate(model, X_train, y_train, criterion)
      val_metrics = evaluate(model, X_val, y_val, criterion)

      if epoch % (params.epochs // 10) == 0:
        print(
            f"Epoch {epoch}/{params.epochs} | "
            f"train_loss={train_loss:.4f} train_acc={train_metrics['accuracy']:.4f} "
            f"train_f1={train_metrics['f_beta_macro']:.4f} "
            f"val_loss={val_metrics['loss']:.4f} val_acc={val_metrics['accuracy']:.4f} "
            f"val_f1={val_metrics['f_beta_macro']:.4f} "
        )

      # Early stopping check
      if val_metrics['loss'] < best_val_loss:
          best_val_loss = val_metrics['loss']
          patience_counter = 0
      else:
          patience_counter += 1
          if patience_counter >= params.patience:
              print(f"Early stopping at epoch {epoch}")
              epochs_run = epoch
              break

  # Return a tuple of fitness values for domination comparison
  # Maximize accuracy, f1, and minimize loss, flops.
  # For loss and flops, we take the negative value.
  val_metrics["flops"] = -estimate_flops(model, (1, 32))

  return (val_metrics["accuracy"], val_metrics["f_beta_macro"], -val_metrics["loss"], -estimate_flops(model, (1, 32)))

In [ ]:
# Test the function
hyperparameters = Hyperparameters(lr=1e-3, epochs=500, hidden=(64, 32, 16, 8), dropout_rate=0.5, patience=100)
# result = train(hyperparameters)

# Neuroevolution

In [11]:
def init_population(pop_size: int) -> List[Hyperparameters]:
    population = []
    for _ in range(pop_size):
        lr = random.uniform(1e-5, 1e-1)
        epochs = random.randint(10, 200)

        # Generate variable-length hidden layer tuple
        num_layers = random.randint(1, 5)
        hidden = tuple(2 ** random.randint(3, 8) for _ in range(num_layers))

        dropout_rate = random.uniform(0.0, 0.5)
        patience = random.randint(5, 30)

        population.append(Hyperparameters(lr, epochs, hidden, dropout_rate, patience))

    return population

In [12]:
def count_dominated(fitnesses: List[Tuple[float, ...]], idx: int) -> int:
    """Count how many points are dominated by fitnesses[idx]"""
    point = fitnesses[idx]
    dominated = 0
    for other in fitnesses:
        if other == point:
            continue
        # Check if point dominates other (all >= and at least one >)
        if all(p >= o for p, o in zip(point, other)) and any(p > o for p, o in zip(point, other)):
            dominated += 1
    return dominated

## Define Evolution Strategy to Optimize Hyperparameters

In [13]:
def evolution_strategy(mu: int, lambda_: int, tau: float, max_gens: int) -> List[Hyperparameters]:
    population = init_population(mu)

    for generation_number in range(1, max_gens + 1):
        print(f"Generation {generation_number} starting initial evaluation")
        fitnesses = []
        for i, member in enumerate(population):
            print(f"Evaluating member {i}: {member}")
            fitnesses.append(train(member))

        # fitnesses = [train(member) for member in population]

        # Calculate proportion of dominated points for each individual
        domination_counts = [count_dominated(fitnesses, i) for i in range(mu)]
        domination_proportions = [count / mu for count in domination_counts]

        offspring = []
        for _ in range(lambda_):
            # Select parent using tournament based on domination proportion
            candidates = random.sample(range(mu), 2)
            parent_idx = max(candidates, key=lambda i: domination_proportions[i])
            parent = population[parent_idx]

            # Mutate hyperparameters
            lr = parent.lr * math.exp(tau * random.gauss(0.0, 1.0))
            lr = max(1e-5, min(1e-1, lr))

            epochs = int(parent.epochs + random.gauss(0.0, 10))
            epochs = max(10, min(500, epochs))

            # Mutate hidden layers
            hidden = list(parent.hidden)
            if random.random() < 0.3:
                if len(hidden) > 1 and random.random() < 0.5:
                    hidden.pop(random.randrange(len(hidden)))
                elif len(hidden) < 5:
                    hidden.insert(random.randrange(len(hidden) + 1), 2 ** random.randint(3, 8))
            else:
                idx = random.randrange(len(hidden))
                hidden[idx] = max(8, min(256, int(hidden[idx] + random.gauss(0.0, 16))))

            dropout_rate = parent.dropout_rate + random.gauss(0.0, 0.05)
            dropout_rate = max(0.0, min(0.5, dropout_rate))

            patience = int(parent.patience + random.gauss(0.0, 3))
            patience = max(5, min(30, patience))

            offspring.append(Hyperparameters(lr, epochs, tuple(hidden), dropout_rate, patience))

        offspring_fitnesses = [train(member) for member in offspring]

        # Calculate domination for offspring
        offspring_domination_counts = [count_dominated(offspring_fitnesses, i) for i in range(lambda_)]
        offspring_domination_proportions = [count / lambda_ for count in offspring_domination_counts]

        # Logging
        best_idx = max(range(mu), key=lambda i: domination_proportions[i])
        print(f"Gen {generation_number} Best: {fitnesses[best_idx]}")

        # Select best member from offspring based on domination
        indexed = [(prop, i) for i, prop in enumerate(offspring_domination_proportions)]
        indexed.sort(key=lambda x: x[0], reverse=True)

        population = [offspring[i] for _, i in indexed[:mu]]

    return population

In [16]:
final_pop = [Hyperparameters(lr=0.075963869926427, epochs=110, hidden=(27, 14), dropout_rate=0.03570794398355265, patience=19),
Hyperparameters(lr=0.058618450698914214, epochs=155, hidden=(49,), dropout_rate=0.24084559091639896, patience=22),
Hyperparameters(lr=0.03591685145376863, epochs=154, hidden=(64,), dropout_rate=0.08112417680099193, patience=18),
Hyperparameters(lr=0.05702871043570519, epochs=150, hidden=(44,), dropout_rate=0.20232812001415448, patience=14),
Hyperparameters(lr=0.07068164533410258, epochs=128, hidden=(34, 22), dropout_rate=0.08438533268973301, patience=23),
Hyperparameters(lr=0.046316062463119444, epochs=167, hidden=(59,), dropout_rate=0.284827195291693, patience=5),
Hyperparameters(lr=0.07101403078911156, epochs=131, hidden=(93,), dropout_rate=0.12339982161939395, patience=28),
Hyperparameters(lr=0.06895945931608811, epochs=111, hidden=(42,), dropout_rate=0.07567568953126667, patience=30),
Hyperparameters(lr=0.05761319230521403, epochs=153, hidden=(58,), dropout_rate=0.2497238481170354, patience=25),
Hyperparameters(lr=0.057957123134657396, epochs=170, hidden=(38,), dropout_rate=0.1719235760065474, patience=16),
Hyperparameters(lr=0.083971988253907, epochs=113, hidden=(53,), dropout_rate=0.17523486601145874, patience=15),
Hyperparameters(lr=0.059138679673085566, epochs=179, hidden=(16, 48), dropout_rate=0.1772226476422656, patience=24),
Hyperparameters(lr=0.04955507050587288, epochs=163, hidden=(63,), dropout_rate=0.22816655253259505, patience=10),
Hyperparameters(lr=0.0788010978205179, epochs=84, hidden=(66,), dropout_rate=0.10675248189853592, patience=6),
Hyperparameters(lr=0.0674894565155647, epochs=153, hidden=(67,), dropout_rate=0.3972397614152903, patience=26),
Hyperparameters(lr=0.07743634327329373, epochs=79, hidden=(32,), dropout_rate=0.08661383231865555, patience=11),
Hyperparameters(lr=0.07491964419750978, epochs=86, hidden=(19, 51), dropout_rate=0.06580738738454688, patience=28),
Hyperparameters(lr=0.05991332160473051, epochs=151, hidden=(86,), dropout_rate=0.5, patience=22),
Hyperparameters(lr=0.08158206009841892, epochs=121, hidden=(34, 55), dropout_rate=0.14618937233485874, patience=23),
Hyperparameters(lr=0.05985280898044459, epochs=158, hidden=(44, 64), dropout_rate=0.23961099900759616, patience=29),
Hyperparameters(lr=0.06016770079064622, epochs=80, hidden=(47,), dropout_rate=0.05991545061780843, patience=27),
Hyperparameters(lr=0.0913161579000739, epochs=67, hidden=(52,), dropout_rate=0.08037962279718079, patience=16),
Hyperparameters(lr=0.07665496489149337, epochs=86, hidden=(50,), dropout_rate=0.17329577809248436, patience=19),
Hyperparameters(lr=0.07202572148217318, epochs=76, hidden=(72,), dropout_rate=0.10964808947852242, patience=6),
Hyperparameters(lr=0.05724451809603519, epochs=168, hidden=(45,), dropout_rate=0.3157578860496876, patience=22),
Hyperparameters(lr=0.07599547323615016, epochs=117, hidden=(40, 68), dropout_rate=0.0954707860264807, patience=9),
Hyperparameters(lr=0.005164234261332292, epochs=228, hidden=(27, 51), dropout_rate=0.19062624499614939, patience=21),
Hyperparameters(lr=0.0806143752346012, epochs=72, hidden=(37,), dropout_rate=0.07246881052498631, patience=11),
Hyperparameters(lr=0.0700278038274355, epochs=115, hidden=(49, 16), dropout_rate=0.12553395358251676, patience=28),
Hyperparameters(lr=0.005563737434721887, epochs=215, hidden=(37, 67), dropout_rate=0.08719107927609329, patience=20),
Hyperparameters(lr=0.07099818060523648, epochs=172, hidden=(54, 45), dropout_rate=0.24697557149746108, patience=26),
Hyperparameters(lr=0.08364310330878642, epochs=117, hidden=(23, 8), dropout_rate=0.08701830180863232, patience=26),
Hyperparameters(lr=0.006795591837295142, epochs=243, hidden=(45, 64), dropout_rate=0.11529955266844638, patience=9),
Hyperparameters(lr=0.06757823290909204, epochs=86, hidden=(20, 77), dropout_rate=0.1562567220599842, patience=23),
Hyperparameters(lr=0.004970784571363729, epochs=214, hidden=(25, 106), dropout_rate=0.12105582693635134, patience=27),
Hyperparameters(lr=0.05166151161353328, epochs=162, hidden=(64, 16, 32), dropout_rate=0.11630147790279065, patience=16),
Hyperparameters(lr=0.05144920483806195, epochs=135, hidden=(48, 42), dropout_rate=0.21032467384703898, patience=15),
Hyperparameters(lr=0.09472557848363745, epochs=84, hidden=(15, 64), dropout_rate=0.10565307869933374, patience=24),
Hyperparameters(lr=0.06480597641772938, epochs=160, hidden=(70, 64), dropout_rate=0.3834200015584367, patience=21),
Hyperparameters(lr=0.06497358378321397, epochs=131, hidden=(35,), dropout_rate=0.24168574633918521, patience=27),
Hyperparameters(lr=0.07031735502681499, epochs=110, hidden=(18, 16, 14), dropout_rate=0.03139186073483591, patience=18),
Hyperparameters(lr=0.04825552853889172, epochs=158, hidden=(54,), dropout_rate=0.2567679607709206, patience=5),
Hyperparameters(lr=0.08396273208128421, epochs=94, hidden=(27, 8), dropout_rate=0.03630625524661061, patience=5),
Hyperparameters(lr=0.04923427010744697, epochs=143, hidden=(46,), dropout_rate=0.21398201853796622, patience=5),
Hyperparameters(lr=0.08065991838707287, epochs=100, hidden=(69,), dropout_rate=0.20408689261716048, patience=9),
Hyperparameters(lr=0.06940133543683716, epochs=106, hidden=(33,), dropout_rate=0.0, patience=16),
Hyperparameters(lr=0.05110468877753123, epochs=163, hidden=(16, 128, 32), dropout_rate=0.15930852353383868, patience=28),
Hyperparameters(lr=0.05796641943260917, epochs=149, hidden=(16, 8), dropout_rate=0.059206445698600244, patience=12),
Hyperparameters(lr=0.07387344004774914, epochs=94, hidden=(44,), dropout_rate=0.23724236072521934, patience=25),
Hyperparameters(lr=0.0723942800422728, epochs=73, hidden=(19, 76), dropout_rate=0.10380150468382528, patience=27),
Hyperparameters(lr=0.08074521180562258, epochs=106, hidden=(27, 64), dropout_rate=0.06880357788570875, patience=9),
Hyperparameters(lr=0.07967962200677163, epochs=96, hidden=(8,), dropout_rate=0.002862267873272907, patience=18),
Hyperparameters(lr=0.06788321735193605, epochs=87, hidden=(19, 60), dropout_rate=0.12008559782307664, patience=27),
Hyperparameters(lr=0.0573499384501241, epochs=151, hidden=(8, 16), dropout_rate=0.0, patience=9),
Hyperparameters(lr=0.07284624614928831, epochs=58, hidden=(40,), dropout_rate=0.0, patience=10),
Hyperparameters(lr=0.07426375969821043, epochs=134, hidden=(11, 40), dropout_rate=0.18820374965742573, patience=18),
Hyperparameters(lr=0.08456981765995479, epochs=145, hidden=(8, 55), dropout_rate=0.20322808964712197, patience=14),
Hyperparameters(lr=0.046309303045632536, epochs=150, hidden=(23, 42), dropout_rate=0.36420054930291346, patience=20),
Hyperparameters(lr=0.06090058137177728, epochs=167, hidden=(39,), dropout_rate=0.5, patience=22),
Hyperparameters(lr=0.03953000836662636, epochs=93, hidden=(51, 23), dropout_rate=0.39069553088377457, patience=26),
Hyperparameters(lr=0.06397309147497401, epochs=87, hidden=(23,), dropout_rate=0.18028264362359903, patience=30),
Hyperparameters(lr=0.0050951023845820535, epochs=200, hidden=(82,), dropout_rate=0.04318069839862673, patience=23),
Hyperparameters(lr=0.06716254757644693, epochs=129, hidden=(8,), dropout_rate=0.07860953792386219, patience=29),
Hyperparameters(lr=0.005235635434359045, epochs=227, hidden=(37,), dropout_rate=0.0, patience=26),
Hyperparameters(lr=0.03246880988921997, epochs=117, hidden=(41, 32), dropout_rate=0.5, patience=5),
Hyperparameters(lr=0.051401368141830536, epochs=131, hidden=(44, 128), dropout_rate=0.3099107758141693, patience=20),
Hyperparameters(lr=0.05858648600352965, epochs=149, hidden=(32, 8), dropout_rate=0.1869890634647153, patience=29),
Hyperparameters(lr=0.07056025138079243, epochs=75, hidden=(8, 64), dropout_rate=0.0, patience=21),
Hyperparameters(lr=0.04975330493306714, epochs=150, hidden=(9, 16), dropout_rate=0.23100845378154644, patience=11),
Hyperparameters(lr=0.06071023420529073, epochs=163, hidden=(20,), dropout_rate=0.5, patience=25),
Hyperparameters(lr=0.05520622275614853, epochs=155, hidden=(19, 8), dropout_rate=0.2564934987446767, patience=11),
Hyperparameters(lr=0.005991239454495941, epochs=246, hidden=(66, 32, 64), dropout_rate=0.05836735890213736, patience=12),
Hyperparameters(lr=0.048952438006446816, epochs=177, hidden=(8, 63), dropout_rate=0.23926640558564682, patience=24),
Hyperparameters(lr=0.08408432906889683, epochs=101, hidden=(20, 8), dropout_rate=0.09442549913575635, patience=15),
Hyperparameters(lr=0.0749659844040317, epochs=80, hidden=(11,), dropout_rate=0.06576117258504678, patience=21),
Hyperparameters(lr=0.09397984553756318, epochs=70, hidden=(53, 64), dropout_rate=0.022279241802995883, patience=17),
Hyperparameters(lr=0.006508156426303994, epochs=256, hidden=(66, 61), dropout_rate=0.15262933637770693, patience=5),
Hyperparameters(lr=0.060268049787930454, epochs=200, hidden=(27, 16), dropout_rate=0.03091522702311223, patience=22),
Hyperparameters(lr=0.0541036134914209, epochs=153, hidden=(13, 35, 16), dropout_rate=0.174647212710617, patience=28),
Hyperparameters(lr=0.0064289472837459165, epochs=222, hidden=(66, 49), dropout_rate=0.11260893711122744, patience=5),
Hyperparameters(lr=0.058528004298667016, epochs=163, hidden=(46, 93), dropout_rate=0.5, patience=29),
Hyperparameters(lr=0.06386016889023398, epochs=165, hidden=(16, 26, 8), dropout_rate=0.4340944047797902, patience=11),
Hyperparameters(lr=0.004660239600601557, epochs=227, hidden=(42, 87), dropout_rate=0.0, patience=20),
Hyperparameters(lr=0.07051373453806209, epochs=124, hidden=(34, 54), dropout_rate=0.0, patience=23),
Hyperparameters(lr=0.07496598755578027, epochs=94, hidden=(32, 20), dropout_rate=0.34924173263005776, patience=28),
Hyperparameters(lr=0.06875038962445905, epochs=113, hidden=(8, 99), dropout_rate=0.13910503576816632, patience=28),
Hyperparameters(lr=0.0768118382630108, epochs=72, hidden=(64, 71), dropout_rate=0.13519642210781616, patience=5),
Hyperparameters(lr=0.05646251441732532, epochs=170, hidden=(10, 256), dropout_rate=0.14381812448297193, patience=18),
Hyperparameters(lr=0.07019020403074706, epochs=102, hidden=(70, 158), dropout_rate=0.10644749136598143, patience=20),
Hyperparameters(lr=0.08493804256424775, epochs=87, hidden=(48, 128), dropout_rate=0.1647923390020374, patience=18),
Hyperparameters(lr=0.07327040119395435, epochs=115, hidden=(54, 128), dropout_rate=0.10316116891003746, patience=17),
Hyperparameters(lr=0.07269762862341113, epochs=109, hidden=(256, 49), dropout_rate=0.06311284549365469, patience=28),
Hyperparameters(lr=0.07472873582379554, epochs=106, hidden=(128, 49), dropout_rate=0.12982785211497452, patience=30),
Hyperparameters(lr=0.05412337655931938, epochs=131, hidden=(128, 44), dropout_rate=0.2836552443611482, patience=13),
Hyperparameters(lr=0.0720624633486621, epochs=120, hidden=(49, 256), dropout_rate=0.1065832816752969, patience=30),
Hyperparameters(lr=0.05140577697147596, epochs=143, hidden=(128, 44), dropout_rate=0.3627274999006924, patience=20),
Hyperparameters(lr=0.1, epochs=104, hidden=(64, 24, 64), dropout_rate=0.2236299180500707, patience=21),
Hyperparameters(lr=0.08548694896021478, epochs=128, hidden=(40, 64, 256), dropout_rate=0.09582517795823102, patience=14),
Hyperparameters(lr=0.005815400886198472, epochs=225, hidden=(66, 64, 256), dropout_rate=0.14997780640643327, patience=16),
Hyperparameters(lr=0.08092507001962815, epochs=103, hidden=(27, 20), dropout_rate=0.030842370447529834, patience=5)]

In [ ]:
final_pop = evolution_strategy(mu=100, lambda_=100, tau=0.05, max_gens=10)
for member in final_pop:
    print(member)

# Test output

In [22]:
# Stats for final generation
final_pop_stats = [train(member) for member in final_pop]
print(final_pop_stats)

Epoch 11/110 | train_loss=0.3107 train_acc=0.8037 train_f1=0.2941 val_loss=2.9974 val_acc=0.7258 val_f1=0.1669 
Epoch 22/110 | train_loss=0.1797 train_acc=0.9320 train_f1=0.3646 val_loss=2.7483 val_acc=0.8419 val_f1=0.1882 
Early stopping at epoch 28
Epoch 15/155 | train_loss=0.2518 train_acc=0.8646 train_f1=0.3968 val_loss=3.2697 val_acc=0.7790 val_f1=0.1856 
Early stopping at epoch 23
Epoch 15/154 | train_loss=0.2198 train_acc=0.8958 train_f1=0.4296 val_loss=3.3257 val_acc=0.8129 val_f1=0.1831 
Early stopping at epoch 19
Epoch 15/150 | train_loss=0.2628 train_acc=0.8738 train_f1=0.3302 val_loss=3.0034 val_acc=0.8081 val_f1=0.1822 
Epoch 30/150 | train_loss=0.1384 train_acc=0.9171 train_f1=0.4718 val_loss=3.5969 val_acc=0.8242 val_f1=0.1902 
Early stopping at epoch 30
Epoch 12/128 | train_loss=0.2942 train_acc=0.7519 train_f1=0.2826 val_loss=2.7488 val_acc=0.6790 val_f1=0.1579 
Epoch 24/128 | train_loss=0.2085 train_acc=0.9057 train_f1=0.3488 val_loss=2.7371 val_acc=0.8210 val_f1=0.18

In [18]:
# graph ouput of multi-objective optimization
import matplotlib.pyplot as plt
import numpy as np



In [19]:
X_test, y_test = prepare_data(test_dataset, device)

# Get predictions
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    predictions = probs.argmax(axis=1)

y_true = y_test.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_true, predictions)
f_beta = fbeta_score(y_true, predictions, average="macro", beta=2)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F-Beta (macro): {f_beta:.4f}")

NameError: name 'prepare_data' is not defined

## Extract Statistics

### Subtask:
Extract the 'val_accuracy', 'val_f1', 'val_loss', and 'flops' from each member's statistics in `final_pop_stats`.


In [25]:
print(final_pop_stats)

[(0.85, 0.18943206326383896, -3.2353885173797607, -1312), (0.8483870967741935, 0.19477075497810123, -3.637916088104248, -1813), (0.8290322580645161, 0.1860296778863554, -3.2799551486968994, -2368), (0.8241935483870968, 0.19023575517257338, -3.5969295501708984, -1628), (0.8403225806451613, 0.188018765788524, -3.8092715740203857, -1946), (0.617741935483871, 0.14546145081655906, -6.4402337074279785, -2183), (0.8580645161290322, 0.19657043208680489, -7.0202460289001465, -3441), (0.8741935483870967, 0.19927892295435334, -6.14325475692749, -1554), (0.8516129032258064, 0.1900647948164147, -4.950568675994873, -2146), (0.8451612903225807, 0.18896501983411468, -2.8774282932281494, -1406), (0.8370967741935483, 0.1872970046914471, -4.563498497009277, -1961), (0.8435483870967742, 0.18867243867243869, -2.4470412731170654, -1520), (0.75, 0.17171344165435748, -5.272684097290039, -2331), (0.5403225806451613, 0.12999611951882034, -11.490469932556152, -2442), (0.8516129032258064, 0.1954912160732021, -4.8

In [24]:
accuracies = []
f1_scores = []
losses = []
flops_values = []

for member_stats in final_pop_stats:
    #if member_stats['val_accuracy'] > 0.1:
    accuracies.append(member_stats['val_accuracy'])
    f1_scores.append(member_stats['val_f1'])
    losses.append(member_stats['val_loss'])
    flops_values.append(member_stats['flops'])

print("Extracted accuracies:", accuracies)
print("Extracted F1 scores:", f1_scores)
print("Extracted losses:", losses)
print("Extracted FLOPs:", flops_values)

TypeError: tuple indices must be integers or slices, not str

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a figure and a 3D axes object
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Create the 3D scatter plot
scatter = ax.scatter(accuracies, f1_scores, losses, c=flops_values, cmap='viridis', s=100, alpha=0.8)

# Label the axes
ax.set_xlabel('Validation Accuracy')
ax.set_ylabel('Validation F1 Score')
ax.set_zlabel('Validation Loss')

# Set the title of the plot
ax.set_title('Hyperparameter Performance in 3D Space (FLOPs as Color)')

# Add a color bar
cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('FLOPs')

# Display the plot
plt.show()

# Task
Prepare fitness data from `final_pop_stats` for Pareto dominance checking, considering accuracy, F1-score, negative loss, and negative FLOPs as objectives. Identify the Pareto front by determining which solutions are non-dominated. Extract the validation accuracy, F1-score, and loss values for these Pareto front solutions. Then, modify the existing 3D scatter plot in cell `072ca572` to visualize these Pareto front solutions with a distinct color and line connections, updating the plot title and legends as needed.

## Prepare Fitness Data for Pareto Front

### Subtask:
Convert the `final_pop_stats` (which are dictionaries) into a list of tuples, where each tuple represents the fitness objectives in a 'greater is better' format. This will include accuracy, F1-score, negative loss, and negative FLOPs, as expected for Pareto dominance checking.


**Reasoning**:
The subtask requires converting the dictionary-based fitness statistics into a list of tuples, applying the 'greater is better' principle by negating loss and FLOPs. This code block will perform the requested conversion and store the result in `fitness_data`.



In [ ]:
fitness_data = []

for member_stats in final_pop_stats:
    accuracy = member_stats['val_accuracy']
    f1_score = member_stats['val_f1']
    loss = member_stats['val_loss']
    flops = member_stats['flops']

    # Negate loss and flops for 'greater is better' comparison
    if accuracy > 0.1:
      fitness_data.append((accuracy, f1_score, -loss, -flops))

print("Prepared fitness data for Pareto front (accuracy, f1_score, -loss, -flops):")
for item in fitness_data:
    print(item)

**Reasoning**:
Now that the fitness data is prepared, the next step is to implement the Pareto dominance check function to identify the non-dominated solutions. This function will be crucial for determining the Pareto front.



In [ ]:
def is_dominated(p1, p2):
    """
    Checks if point p1 is dominated by point p2.
    A point p1 is dominated by p2 if p2 is better than or equal to p1 in all objectives,
    and strictly better in at least one objective.
    Assumes 'greater is better' for all objectives.
    """
    # Check if p2 is strictly better in at least one objective
    strictly_better = False
    for i in range(len(p1)):
        if p2[i] > p1[i]:
            strictly_better = True
        elif p2[i] < p1[i]: # p2 is worse than p1 in this objective, so p1 is not dominated by p2
            return False
    return strictly_better

def find_pareto_front(points):
    """
    Finds the Pareto front from a list of points (solutions).
    """
    pareto_front = []
    for i, p1 in enumerate(points):
        is_p1_dominated = False
        for j, p2 in enumerate(points):
            if i == j:
                continue
            if is_dominated(p1, p2):
                is_p1_dominated = True
                break
        if not is_p1_dominated:
            pareto_front.append(p1)
    return pareto_front

# Find the Pareto front using the prepared fitness data
pareto_solutions = find_pareto_front(fitness_data)

print("Pareto Front Solutions:")
for solution in pareto_solutions:
    print(solution)

**Reasoning**:
Extract the accuracy, F1-score, and positive loss values from each Pareto front solution to prepare them for plotting.



In [ ]:
pareto_accuracies = []
pareto_f1_scores = []
pareto_losses = []

for solution in pareto_solutions:
    # solution is (accuracy, f1_score, -loss, -flops)
    pareto_accuracies.append(solution[0])
    pareto_f1_scores.append(solution[1])
    pareto_losses.append(-solution[2]) # Negate back to positive loss

print("Pareto Front Accuracies:", pareto_accuracies)
print("Pareto Front F1 Scores:", pareto_f1_scores)
print("Pareto Front Losses:", pareto_losses)

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot all initial solutions as before
scatter = ax.scatter(accuracies, f1_scores, losses, c=flops_values, cmap='viridis', s=100, alpha=0.6, label='All Solutions')

# Create a triangulated surface for the Pareto front
# plot_trisurf can handle unsorted points and triangulate them
ax.plot_trisurf(pareto_accuracies, pareto_f1_scores, pareto_losses, color='orange', alpha=0.5, label='Pareto Front Surface')

# Highlight the individual Pareto front solutions with distinct scatter points
ax.scatter(pareto_accuracies, pareto_f1_scores, pareto_losses, color='black', s=200, marker='o', edgecolors='black', label='Pareto Front Solutions', zorder=10)

# Label the axes
ax.set_xlabel('Validation Accuracy')
ax.set_ylabel('Validation F1 Score')
ax.set_zlabel('Validation Loss')

# Set the title of the plot
ax.set_title('Hyperparameter Performance (FLOPs as Color) with Pareto Front Surface')

# Add a color bar for the scatter plot (all solutions) and set its label to 'FLOPs'
cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('FLOPs')

# Add legend to distinguish all solutions from Pareto front solutions/surface
# The label from plot_trisurf might not appear in legend directly, so we explicitly add 'Pareto Front Solutions'
# For a true surface legend, a custom legend handle might be needed, but scatter for points is sufficient as requested.
ax.legend()

# Display the plot
plt.show()